# Hamilton Markov-Switching Model with EM

Same train/test split as [testingRNN.ipynb](testingRNN.ipynb). Fits Hamilton's (1989) Markov-switching AR(4) via the **EM algorithm** (E-step: Hamilton filter + Kim smoother; M-step: closed-form updates). Produces filtered regime probabilities and one-step-ahead fitted values.

**Dependency:** `pip install statsmodels`

## 1. Data — Same Split as testingRNN

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Same as testingRNN
training_data = pd.read_csv('./gnp_data_created_cpp.csv', low_memory=False)
dy = training_data['dy'].astype(np.float64)

WINDOW_SIZE = 4
split_idx = int(len(dy) * 0.8)
train_dy = dy.iloc[:split_idx]
val_dy = dy.iloc[split_idx - WINDOW_SIZE:]  # overlap for valid first X

print(f"Train: {len(train_dy)} points | Val: {len(val_dy)} points (with {WINDOW_SIZE} overlap)")
print(f"Validation starts at index {split_idx}")

Train: 800000 points | Val: 200005 points (with 4 overlap)
Validation starts at index 800000


## 2. EM-Style Estimation: Hamilton Filter + Kim Smoother

statsmodels `MarkovAutoregression` uses **EM** for parameter estimation (`em_iter` steps) before optional BFGS polish. We set `em_iter=100` to emphasize EM. The model matches the DGP: AR(4) with regime-switching means (Hamilton 1989).

In [ ]:
from statsmodels.tsa.regime_switching.markov_autoregression import MarkovAutoregression

# Hamilton (1989): AR(4) with switching mean only (switching_ar=False)
mod = MarkovAutoregression(
    train_dy,
    k_regimes=2,
    order=4,
    switching_ar=False,
)
res = mod.fit(
    em_iter=100,   # EM steps (EM-style estimation)
    maxiter=50,    # BFGS polish after EM
    disp=True,
)
print(res.summary())

In [ ]:
# Filtered regime probabilities P(S_t=0 | y_1:t) = P(recession | y_1:t)
filter_res = mod.filter(res.params)
P_recession_train = filter_res.filtered_marginal_probabilities[0]  # regime 0 = recession
print(f"Filtered P(recession) on train: shape {P_recession_train.shape}")

## 3. One-Step-Ahead Forecasts on Validation

Use `predict` with `probabilities='predicted'` (P(S_t | y_1:t-1)) to get one-step-ahead forecasts. Model is fit on train; we run the filter on full series with fixed params to get val forecasts.

In [ ]:
# One-step-ahead: run filter on full series with fitted params, use 'predicted' probs
mod_full = MarkovAutoregression(dy, k_regimes=2, order=4, switching_ar=False)
pred_full = mod_full.predict(
    res.params,
    start=split_idx,
    end=len(dy) - 1,
    probabilities='predicted',  # P(S_t | y_1:t-1) for one-step-ahead
)
kalman_preds = np.asarray(pred_full).flatten()
Y_val_true = val_dy.iloc[WINDOW_SIZE:].values  # exclude overlap
print(f"One-step-ahead predictions: {len(kalman_preds)} validation points")

In [ ]:
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

mse_kalman_val = mse(Y_val_true, kalman_preds)
print(f"Hamilton EM (validation) MSE: {mse_kalman_val:.6f}")

# In-sample fitted values on train
train_preds = res.predict(probabilities='filtered')
Y_train_true = train_dy.iloc[WINDOW_SIZE:].values
train_preds_trimmed = np.asarray(train_preds.iloc[WINDOW_SIZE:]).flatten()
mse_kalman_train = mse(Y_train_true, train_preds_trimmed)
print(f"Hamilton EM (train) MSE: {mse_kalman_train:.6f}")

In [ ]:
# Plot: actual vs Hamilton one-step-ahead on validation
fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True)
n_plot = min(500, len(Y_val_true))

axes[0].plot(Y_val_true[:n_plot], label='Actual', alpha=0.8)
axes[0].plot(kalman_preds[:n_plot], label='Hamilton 1-step-ahead', alpha=0.8)
axes[0].set_ylabel('dy (GNP growth)')
axes[0].legend()
axes[0].set_title('Hamilton EM: fitted values on validation')

# Filtered P(recession) on last part of train (if we have it from extended model)
n_train_plot = min(500, len(P_recession_train))
axes[1].plot(P_recession_train[-n_train_plot:], color='gray', label='P(recession) filtered')
axes[1].set_ylabel('Probability')
axes[1].set_xlabel('Time (train)')
axes[1].legend()
axes[1].set_ylim(0, 1)
plt.tight_layout()
plt.show()

## 4. Comparison with testingRNN Baselines

Run testingRNN to get `mse_mean_val`, `mse_ar4_val`, `mse_nn_val`. Compare with `mse_kalman_val` above.

| Model | Val MSE |
|-------|--------|
| Predict mean | (from testingRNN) |
| AR(4) OLS | (from testingRNN) |
| Hamilton EM | (above) |
| Neural Net | (from testingRNN) |